# Retrieval of data to put in the manuscript text/tables

In [ ]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
# Location of reemission outputs
outputs = pathlib.Path("outputs/reemission/outputs_MIN_LOW_PRIM.xlsx")
reemission_inputs = pd.read_excel(outputs,'inputs')
reemission_outputs = pd.read_excel(outputs,'outputs')
reemission_outputs['total_net'] = reemission_outputs['co2_net'] + reemission_outputs['ch4_net']
hp_inputs = reemission_inputs.query('type=="hydroelectric"')
irr_inputs = reemission_inputs.query('type=="irrigation"')
multi_inputs = reemission_inputs.query('type=="multipurpose"')
hp_names = hp_inputs['Name'].to_list()
irr_names = irr_inputs['Name'].to_list()
multi_names = multi_inputs['Name'].to_list()
hp_mask = reemission_outputs['Name'].isin(hp_names)
irr_mask = reemission_outputs['Name'].isin(irr_names)
multi_mask = reemission_outputs['Name'].isin(multi_names)
hp_outputs = reemission_outputs[hp_mask]
irr_outputs = reemission_outputs[irr_mask]
multi_outputs = reemission_outputs[multi_mask]
# MOO inputs
moo_inputs_path = pathlib.Path("outputs/moo/all_hp.csv")
# SHAPE WITH EMISSION INTENSITIES
shp_path = pathlib.Path("intermediate/out_par_em_ifc.shp")

In [ ]:
reemission_outputs.head()

In [ ]:
reemission_outputs['total_net'].describe()

## Data needed for the Results section, aerial emission rates

In [ ]:
reemission_outputs.columns

In [ ]:
reemission_inputs.head()

## Hydroelectric reservoirs

In [ ]:
# Areas in km2
hp_inputs['res_area'].describe()

In [ ]:
# Provide volumes in km3
(hp_inputs['res_volume']/1e9).describe()

In [ ]:
hp_inputs['res_mean_depth'].describe()

In [ ]:
hp_outputs['co2_net'].describe()

In [ ]:
hp_outputs['ch4_net'].describe()

In [ ]:
hp_outputs['total_net'].describe()

## Irrigation reservoirs

In [ ]:
irr_inputs['res_area'].describe()

In [ ]:
# Provide volumes in km3
(irr_inputs['res_volume']/1e9).describe()

In [ ]:
# sorted(irr_inputs['res_area'].to_list())
irr_inputs['res_mean_depth'].describe()

In [ ]:
irr_outputs['total_net'].describe()

In [ ]:
irr_outputs['co2_net'].describe()

In [ ]:
irr_outputs['ch4_net'].describe()

In [ ]:
irr_inputs['res_mean_depth'].hist(bins=100)

## Multipurpose reservoirs

In [ ]:
multi_inputs['res_area'].describe()

In [ ]:
# Provide volumes in km3
(multi_inputs['res_volume']/1e9).describe()

In [ ]:
# sorted(irr_inputs['res_area'].to_list())
multi_inputs['res_mean_depth'].describe()

In [ ]:
multi_outputs['total_net'].describe()

In [ ]:
multi_outputs['co2_net'].describe()

In [ ]:
multi_outputs['ch4_net'].describe()

## Data needed for the Emission intensities of existing and planned hydropower section

In [ ]:
moo_inputs = pd.read_csv(moo_inputs_path)
# units:
# HP_mean in MW
# tot_em in tonnesCO2/anum
# em_intensity in gCO2/kWh 

moo_inputs_future_sto = moo_inputs.query('type=="hydroelectric" & status=="Future" & hp_type_ifc =="S"')
moo_inputs_existing_sto = moo_inputs.query('type=="hydroelectric" & status=="Existing" & hp_type_ifc =="S"')
moo_inputs_future_multi = moo_inputs.query('type=="multipurpose" & status=="Future" & hp_type_ifc =="S"')
moo_inputs_existing_multi = moo_inputs.query('type=="multipurpose" & status=="Existing" & hp_type_ifc =="S"')

moo_inputs_future = moo_inputs.query('status=="Future" & hp_type_ifc =="S"')
moo_inputs_existing = moo_inputs.query('status=="Existing" & hp_type_ifc =="S"')

def calc_em_intensity(data):
    return data['tot_em'] * 1000000 / (data['HP_mean'] * 24 * 365.25 * 1000) 

def calc_em(data):
    return data['total_net']

In [ ]:
calc_em_intensity(data=moo_inputs_future_sto).describe()

In [ ]:
calc_em_intensity(data=moo_inputs_existing_sto).describe()

In [ ]:
calc_em_intensity(moo_inputs_future_multi).describe()

In [ ]:
calc_em_intensity(moo_inputs_existing_multi).describe()

In [ ]:
calc_em_intensity(moo_inputs_future).describe()

In [ ]:
calc_em_intensity(moo_inputs_existing).describe()

In [ ]:
#calc_em(moo_inputs_existing).describe()

In [ ]:
moo_inputs.query('status=="Future"')["HP_mean"].sum()

In [ ]:
moo_inputs.query('status=="Existing"')["HP_mean"].sum()

In [ ]:
moo_inputs.query('type=="multipurpose"')

In [ ]:
moo_inputs.head()

In [ ]:
moo_inputs.query('status == "Future" and hp_type_ifc == "RoR"').count()

In [ ]:
moo_inputs.query('status == "Existing" and hp_type_ifc == "RoR"').count()

In [ ]:
moo_inputs.query('type == "hydroelectric" and status == "Existing" and hp_type_ifc == "S"').count()

In [ ]:
moo_inputs.query('type == "multipurpose" and status == "Existing" and hp_type_ifc == "S"').count()

### Find emission intensities

In [ ]:
gdf = gpd.read_file(shp_path).rename(columns={'reservoir_': 'res_type'})

In [ ]:
gdf.columns

In [ ]:
gdf['status']

In [ ]:
print(gdf[['hp_type_re', 'em_intensi']])

In [ ]:
gdf.query('hp_type_re != "ror"')['em_intensi'].describe()

In [ ]:
gdf_trimmed = gdf[gdf['em_intensi']>0.6]

In [ ]:
gdf_trimmed.query('hp_type_re != "ror"')['em_intensi'].describe()

In [ ]:
gdf.query('res_type == "multipurpose"')['em_intensi'].describe()

In [ ]:
gdf[(gdf.ro_r_or_st == 'S') & (gdf.res_type == 'hydroelectric')]['em_intensi'].describe()

In [ ]:
gdf.query('res_type == "multipurpose"')['des_head_1'].astype(float).describe()

In [ ]:
gdf[(gdf.ro_r_or_st == 'S') & (gdf.res_type == 'hydroelectric')]['des_head_1'].astype(float).describe()

In [ ]:
hp_sto = gdf[(gdf.ro_r_or_st == 'S') & (gdf.res_type == 'hydroelectric')]
hp_sto.query('status == "Built"')['em_intensi'].describe()

In [ ]:
hp_sto.query('status != "Built"')['em_intensi'].describe()

In [ ]:
gdf[gdf['index'] == "Lemro 2"]['em_intensi']

In [ ]:
gdf[gdf['index'] == "Belin"]['em_intensi']

In [ ]:
gdf[gdf['index'] == "Laza"]['em_intensi']

### Hydropower generation

In [ ]:
current_gen = gdf.query('status == "Built"')['ann_gen'].sum()

In [ ]:
planned_gen = gdf.query('status != "Built"')['ann_gen'].sum()

In [ ]:
current_gen, planned_gen

In [ ]:
current_gen + planned_gen

In [ ]:
current_gen / (current_gen + planned_gen)

### Optimization

In [ ]:
nondom = pd.read_csv('em_int_nondom_df.csv')

In [ ]:
nobuilt = nondom[nondom['Scenario, [1/0]']==0]
built = nondom[nondom['Scenario, [1/0]']==1]

In [ ]:
nobuilt[nobuilt['Mean annual HP, [MW]']>1814].head(1)

In [ ]:
built.head(1)